In [1]:
import pandas as pd 
import numpy as np 
import datetime

In [2]:
df15mar_15apr = pd.read_csv('H:\intern_lovelocal/sab_kuch_milega.csv')
df = df15mar_15apr[['customer_id','created_on']]

In [3]:
#CREATING DATE TIME OBJECT
start_date_obj= datetime.datetime.strptime('2020-02-01', '%Y-%m-%d').date()
end_date_obj= datetime.datetime.strptime('2020-04-15', '%Y-%m-%d').date()

In [5]:
def filt_by_date(start_date_obj, end_date_obj, df):
       mask = ((df['Dates'] > start_date_obj) & (df['Dates'] <= end_date_obj))
       df = df.loc[mask]
       return df

In [6]:
#PREPROCESSING THE DATA FRAME
def prePro_df1(start_date_obj, end_date_obj, df15mar_15apr_slc_col):
      df15mar_15apr_slc_col['created_on']= pd.to_datetime(df15mar_15apr_slc_col['created_on'])
      df15mar_15apr_slc_col['created_on'].dt.tz_localize(None)
      #TypeError: Cannot compare tz-naive and tz-aware datetime-like objects, HENCE SPLITTING DATE AND TIME
      df15mar_15apr_slc_col['Dates'] = pd.to_datetime(df15mar_15apr_slc_col['created_on']).dt.date
      df15mar_15apr_slc_col = df15mar_15apr_slc_col[['customer_id','Dates']]
      #CALLING FILT BY DATE 
      df15mar_15apr_slc_col = filt_by_date(start_date_obj, end_date_obj, df15mar_15apr_slc_col)
      return df15mar_15apr_slc_col

In [7]:
#CALLING PREpRO_DF1
df15mar_15apr_slc_col= prePro_df1(start_date_obj, end_date_obj, df)

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
#RESETTING INDEX
df15mar_15apr_slc_col = df15mar_15apr_slc_col.sort_values(by=['Dates'])#FUNCTION TO FILTER DATA BY DATE
df15mar_15apr_slc_col.reset_index(inplace= True)#RESET INDEX
df15mar_15apr_slc_col = df15mar_15apr_slc_col[['customer_id','Dates']]#REMOVING OLD INDEX COLUMN


In [9]:

#################################################################################################
#FINDING CUSTOMERS WHO ORDERED MORE THAN 1 TIMES
#REMOVING CUSTOMERS ORDERED ON SAME DATES 
df15mar_15apr_slc_col = df15mar_15apr_slc_col.drop_duplicates(subset=['customer_id', 'Dates'], keep='first')
#KEEPING ONLY DUPLICATED CUSTOMER IDS
df15mar_15apr_slc_col_dups = df15mar_15apr_slc_col[df15mar_15apr_slc_col.duplicated(subset=["customer_id"], keep=False)]
#CALCULATING FREQUENCY BY PIVOTING
df15mar_15apr_slc_col_dups = df15mar_15apr_slc_col_dups.pivot_table(index=['customer_id'], aggfunc='size').sort_values(ascending= False)
#Converting from series to dataframe
df15mar_15apr_slc_col_dups = df15mar_15apr_slc_col_dups.to_frame()
#NAming third column
df15mar_15apr_slc_col_dups = df15mar_15apr_slc_col_dups.rename(columns = {0: "freq"})
#Setting the Index
df15mar_15apr_slc_col_dups.reset_index(inplace= True)

In [10]:
print(df15mar_15apr_slc_col)

                            customer_id       Dates
0      b31f89664e91417d8bda81c02dee3fc4  2020-02-02
1      f2bb1de2d52c40e5a165b928dd6fefca  2020-02-02
2      dae1f7beee494b13975b54b6f5c23fde  2020-02-02
3      417e83c08db243f5b976cf42039cb452  2020-02-02
4      dedb8158519e448eb8680619b0871368  2020-02-02
...                                 ...         ...
20667  9c5eb857da0244579dfb68ebd9ac79a5  2020-04-15
20668  7d7e5e68e8cb45629fc4a22e3de82c79  2020-04-15
20672  72448b0aeaf34b5d8159dbe1306f6eda  2020-04-15
20675  e9c5ac2906b14ed6b044ee0a763c9555  2020-04-15
20677  6fdbc679025e498bb4d27d5e4204bfc5  2020-04-15

[18585 rows x 2 columns]


#TOTAL NUMER OF ORDERS FROM 16TH MARCH - 15TH APRIL

In [11]:
print(df15mar_15apr_slc_col_dups)

                           customer_id  freq
0     5c15fb165d8a4afc8c1dad7850104056    20
1     c0f42047fe384b199c7fb216da5f1c25    16
2     0b2e2cb5801144eca4a8c7c8cd1673a2    15
3     1d3bb8e528c949bb9bcb191abb0c1dd5    13
4     d61ebf98b24248779f85bd11f1da68fb    11
...                                ...   ...
2190  9a5acb8e30fb4bf98a953eb8e52fb468     2
2191  9a9ff7f0622e4be7a20b4ba843d76544     2
2192  9aaf21e83a1645b9b4963245f45ae353     2
2193  9ab72d6a52144165a2f3cd928e1ddb2e     2
2194  0011bf17a68041c587cf7b3c9e09285d     2

[2195 rows x 2 columns]


#CUSTOMERS WITH ORDERS > 1. FROM 16TH MARCH - 15TH APRIL

In [12]:
#FINDING CUSTOMERS WHO ORDERED EXACTLY 1 TIMES
#CALCULATING FREQUENCY BY PIVOTING
cust_ordered_1_time = df15mar_15apr_slc_col.pivot_table(index=['customer_id'], aggfunc='size').sort_values(ascending= False)
#Converting from series to dataframe
cust_ordered_1_time = cust_ordered_1_time.to_frame()
#NAming third column
cust_ordered_1_time = cust_ordered_1_time.rename(columns = {0: "freq"})
#Setting the Index
cust_ordered_1_time = cust_ordered_1_time.loc[cust_ordered_1_time['freq'] == 1] 
cust_ordered_1_time.reset_index(inplace= True)


In [13]:
print(cust_ordered_1_time)

                            customer_id  freq
0      53f296df031c4c41bd0901bbc70d75f5     1
1      5394495cd16d4c378a1b1f145cf379f6     1
2      4800c98ba2b945cda59463107f21e226     1
3      541e326b151646e085c40fdb36bf65ba     1
4      480d910f91194a23bc83979ed15435e8     1
...                                 ...   ...
12899  a56ae3900dfd4db19e9857920721efba     1
12900  a556ab40eac742368bcec95025c6fc7d     1
12901  a5568fbf953e41fb867a91ec447d93ec     1
12902  a5490f375cab4fb2bb029d39105ec1ce     1
12903  0001370ecc044bbdaf598d8c229fe42b     1

[12904 rows x 2 columns]


#CUSTOMERS WITH EXACTLY 1 ORDER FROM 16TH MARCH TO 15TH APRIL

In [14]:
#AMONG FRQUENT CUSTOMERS FINDING WHICH CUSTOMERS ARE NOT ORDERING NOW FREQUENTLY

#GIVING INPUT FOR SELECTED DATE 
import copy
df2 = copy.deepcopy(df)


df2['created_on']= pd.to_datetime(df2['created_on'])
df2['created_on'].dt.tz_localize(None)
#TypeError: Cannot compare tz-naive and tz-aware datetime-like objects, HENCE SPLITTING DATE AND TIME
df2['Dates'] = pd.to_datetime(df2['created_on']).dt.date
df2 = df2[['customer_id','Dates']]

start_date_obj= datetime.datetime.strptime('2020-04-15', '%Y-%m-%d').date()
mask = (df2['Dates'] > start_date_obj)
df2 = df2.loc[mask]

#RESETTING INDEX
df2 = df2.sort_values(by=['Dates'])#FUNCTION TO FILTER DATA BY DATE
df2.reset_index(inplace= True)#RESET INDEX
df2 = df2[['customer_id','Dates']]#REMOVING OLD INDEX COLUMN

################################################################################################
      
listt=[]
for i in df15mar_15apr_slc_col_dups['customer_id']:
    if i not in df2.values:
       listt.append(i)       

In [15]:
print(len(listt))

2055


#NUMBER OF CUSTOMERS WHO ARE NOT ORDERING AFTER 15TH APRIL TILL TODAY

#CUSTOMERS WITH > 1 ORDER FROM 16TH MARCH TO 15TH IS 859, AMONG THOSE 859 CUSTOMERS, 769 OF CUSTOMERS HAVE 
#NOT PLACED ANY ORDER AFTER 15TH OF APRIL

In [16]:
#CONSIDERING MORE THAN 2 ORDERS
rslt_df = df15mar_15apr_slc_col_dups.loc[df15mar_15apr_slc_col_dups['freq'] >= 1] 
 
listt2=[]
for i in rslt_df['customer_id']:
    if i not in df2.values:
       listt2.append(i)
       

In [17]:
print(rslt_df)

                           customer_id  freq
0     5c15fb165d8a4afc8c1dad7850104056    20
1     c0f42047fe384b199c7fb216da5f1c25    16
2     0b2e2cb5801144eca4a8c7c8cd1673a2    15
3     1d3bb8e528c949bb9bcb191abb0c1dd5    13
4     d61ebf98b24248779f85bd11f1da68fb    11
...                                ...   ...
2190  9a5acb8e30fb4bf98a953eb8e52fb468     2
2191  9a9ff7f0622e4be7a20b4ba843d76544     2
2192  9aaf21e83a1645b9b4963245f45ae353     2
2193  9ab72d6a52144165a2f3cd928e1ddb2e     2
2194  0011bf17a68041c587cf7b3c9e09285d     2

[2195 rows x 2 columns]


In [18]:
print(len(listt2))

2055


#### Total customers with > 2 orders from 16th march - 15th april is "252"

In [19]:
Customers_not_ordering = pd.DataFrame(listt2) 

In [20]:
Customers_not_ordering

,0
0,0b2e2cb5801144eca4a8c7c8cd1673a2
1,1d3bb8e528c949bb9bcb191abb0c1dd5
2,d61ebf98b24248779f85bd11f1da68fb
3,9ff639829b95435d867c272e2f93ae2e
4,5760d12a58db418cbae5d11a79c7cdfd
...,...
2050,9a5acb8e30fb4bf98a953eb8e52fb468
2051,9a9ff7f0622e4be7a20b4ba843d76544
2052,9aaf21e83a1645b9b4963245f45ae353
2053,9ab72d6a52144165a2f3cd928e1ddb2e


In [21]:
Customers_not_ordering = Customers_not_ordering.rename(columns = {0: "customer_id"})

In [22]:
Customers_not_ordering.to_excel(r'H:\intern_lovelocal\customers_not_ordering_new.xlsx', index = False)

#AMONG THOSE 252 CUSTOMERS NUMBER OF CUSTOMERS WITH MORE THAN 2 ORDERS IN PREV MONTH BUT DID NOT PLACED ANY ORDER AFTER 15TH APRIL

#CUSTOMERS WITH > 2 ORDER FROM 16TH MARCH TO 15TH IS 859, AMONG THOSE 859 CUSTOMERS, 769 OF CUSTOMERS HAVE 
#NOT PLACED ANY ORDER AFTER 15TH OF APRIL

# DID SEPARATELY FOR CUSTOMERS WITH  > 2 AND > 1 ORDERS, BECAUSE, CUSTOMERS WITH EXATLY 2 ORDERS FROM 16TH MARCH - 15TH APRIL  ARE = NEARLY  600 (BIG AMOUNT)  FROM TOTAL OF JUST 859 CUSTOMERS.
# HENCE CALCULATING FOR CUSTOMERS > 2 ORDERS FROM 16MAR- 15APRIL BUT NOT PLCAING ORDERS AFTER THAT BECOMES THE NECESSITY FOR THE PROPER RESULTS AND FURTHER WORK  

In [23]:
customer_not_ordering_latest = pd.merge(left=Customers_not_ordering, right=rslt_df, how='left', left_on='customer_id', right_on='customer_id')

In [24]:
customer_not_ordering_latest.head(10).style.background_gradient(cmap='Blues')

,customer_id,freq
0,0b2e2cb5801144eca4a8c7c8cd1673a2,15
1,1d3bb8e528c949bb9bcb191abb0c1dd5,13
2,d61ebf98b24248779f85bd11f1da68fb,11
3,9ff639829b95435d867c272e2f93ae2e,10
4,5760d12a58db418cbae5d11a79c7cdfd,9
5,b7899cef6022472a9cf38ed256c57e07,9
6,184755e7ee4943e5b48c7086980366ed,9
7,b3fda2de14874fda80b3ef7a091f88e9,9
8,c3c553212956466f810f213dc83a8eef,8
9,47e428d461fd41a081336c5bd2bcfc66,8


In [25]:
customer_not_ordering_latest

,customer_id,freq
0,0b2e2cb5801144eca4a8c7c8cd1673a2,15
1,1d3bb8e528c949bb9bcb191abb0c1dd5,13
2,d61ebf98b24248779f85bd11f1da68fb,11
3,9ff639829b95435d867c272e2f93ae2e,10
4,5760d12a58db418cbae5d11a79c7cdfd,9
...,...,...
2050,9a5acb8e30fb4bf98a953eb8e52fb468,2
2051,9a9ff7f0622e4be7a20b4ba843d76544,2
2052,9aaf21e83a1645b9b4963245f45ae353,2
2053,9ab72d6a52144165a2f3cd928e1ddb2e,2


In [141]:
customer_not_ordering_latest.to_excel(r'H:\intern_lovelocal\customer_not_ordering_latest.xlsx', index = False)